In [ ]:
import numpy as np
import pandas as pd
import datetime
import os
os.chdir("SleepEDF")
Labels=np.load("Labels.npy")
Epochs=np.load("Epochs.npy")

In [ ]:
Labels=Labels.astype(int)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
cuda = torch.device('cuda')
Labels=torch.from_numpy(Labels)
Epochs=torch.from_numpy(Epochs)
Epochs=Epochs.unsqueeze(1)

In [ ]:
model=nn.Sequential(
    nn.Conv1d(1,50,70),
    nn.ReLU(),
    nn.MaxPool1d(3),
    nn.Conv1d(50,50,70),
    nn.ReLU(),
    nn.MaxPool1d(4),
    nn.Conv1d(50,10,11),
    nn.ReLU(),
    nn.MaxPool1d(2),
    nn.Linear(108,1),
)

In [ ]:
model=model.cuda()

In [ ]:
len(Epochs[0])

In [ ]:
FoldSize=len(Epochs)//5

In [ ]:
Folds=[]

In [ ]:
for x in range(5):
    Folds.append(torch.arange(FoldSize*x,FoldSize*(x+1)))

In [ ]:
Folds

In [ ]:
train_Epochs=Epochs[torch.cat((Folds[0],Folds[1],Folds[2],Folds[3]),0)]

In [ ]:
train_Labels=Labels[torch.cat((Folds[0],Folds[1],Folds[2],Folds[3]),0)].cuda()

In [ ]:
test_Epochs=Epochs[Folds[4]]
test_Labels=Labels[Folds[4]]

In [ ]:
Epochs.shape

In [ ]:
np.shape(Labels)

In [ ]:
def training_loop(model):
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    n_batches=84
    batchsize=int(len(train_Epochs)/n_batches)
    for ep in range(1,17):
        loss_v = 0.0
        for i in range(n_batches):
            torch.cuda.empty_cache()
            epoch_Batch=train_Epochs[i*batchsize:(i+1)*batchsize,]
            label_Batch=train_Labels[i*batchsize:(i+1)*batchsize,]
            label_Batch=label_Batch.long()
            label_Batch=label_Batch.unsqueeze(1)
            epoch_Batch=epoch_Batch.float()
            train_t = model(epoch_Batch.cuda())
            loss_train = nn.CrossEntropyLoss()
            outTrain=loss_train(train_t,label_Batch)
            optimizer.zero_grad()
            outTrain.backward()
            optimizer.step()
            loss_v+=outTrain.item()
        print('{} Epoch {}, Training loss {}'.format(
        datetime.datetime.now(), ep,
        loss_v/n_batches))   

In [ ]:
training_loop(model)